In [ ]:
from spotipy.oauth2 import SpotifyClientCredentials
from IPython.display import IFrame
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
import spotipy
import json
import config
import pprint
import textdistance
import random
import numpy as np
import pickle
import matplotlib.pyplot as plt


Hot100_df = pd.read_csv('Hot100.csv')

def initialize_spotify():
    return spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=config.client_id, client_secret=config.client_secret))

def load(filename="newscaler.pickle"):
    try:
        with open(filename, "rb") as f:
            return pickle.load(f)
    except FileNotFoundError:
        print("File not found!")

def play_song(track_id):
    return IFrame(src="https://open.spotify.com/embed/track/"+track_id,
                  width="320",
                  height="80",
                  frameborder="0",
                  allowtransparency="true",
                  allow="encrypted-media",
                  )

def spotify_recommender():
    sp = initialize_spotify()
    
    Hot100_df['songs_lower'] = Hot100_df['songs'].str.lower()

    while True:
        user_input = input("Please write a song that you like: ").lower()
        similarity_scores = Hot100_df['songs_lower'].apply(lambda x: textdistance.jaccard(user_input, x))
        best_match_index = similarity_scores.idxmax()
        best_match_score = similarity_scores.max()
        threshold = 0.6

        if best_match_score >= threshold:
            best_match_song = Hot100_df.loc[best_match_index, 'songs']
            print(f"Did you mean: {best_match_song}?")
            recommended = random.choice(Hot100_df['songs'])
            results = sp.search(q=recommended)
            track_id = results["tracks"]["items"][0]["id"]
            print(f"Cool! You may also like this one:")
            display(play_song(track_id))
            answer = input("\nShould I recommend you another song? [yes/no] ").strip().lower()
            if answer == 'no':
                print("\n\nBye bye.... have fun <o/")
                break
                 
            while answer not in ["yes","no"]:
                print("Sorry! Please enter a valid answer. Try again.")
                answer = input("\nShould I recommend you another song? [yes/no] ").strip().lower()  
                
            if recommended == user_input:
                print(f"Oooops! Let me check another song for you...")
                recommended = Hot100_df['songs'].sample(n=1).values[0]
                print(f"What about this one: {recommended}")
                answer = input("\nShould I recommend you another song? [yes/no] ").strip().lower() 
                
        else:
            print("\nSorry!!!! This song is not on the 'Hot 100 list'.")           
            results = sp.search(q = user_input, limit=5,market="GB")
            print(f"If you like {user_input} you may also like: ")
            
            input_all_features = sp.audio_features(results["tracks"]["items"][0]["id"])[0]
            selected_features = ['danceability', 'energy','loudness', 'speechiness',
                     'acousticness', 'instrumentalness', 'liveness','tempo']
            input_features = {feature: input_all_features[feature] for feature in selected_features}
            input_df = pd.DataFrame([input_features]) 
            newscaler = load("newscaler.pickle")
            scaled_features = newscaler.transform(input_df)
            scaled_features_df = pd.DataFrame(scaled_features, columns = input_df.columns)
            kmeans = load(filename = "spotify_model.pickle")
            predicted_cluster = kmeans.predict(scaled_features_df)[0]
            X = pd.read_csv('new_df.csv')
            
            new_recommended = X[X['cluster'] == predicted_cluster].sample()
            id_= new_recommended["id"].values[0]

            display(play_song(id_))
            answer = input("\nShould I recommend you another song? [yes/no] ").strip().lower()
            
            if answer == 'no':
                print("\n\nBye bye.... have fun <o/")
                break
                
            while answer=="yes":
                spotify_recommender() 
 
            break

In [ ]:
# To run the program:

spotify_recommender()

--------------------------------------------------------------------------------------------------------------